# Scrape

### Mars News

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [29]:
html = browser.html
soup = bs(html, 'html.parser')

# Search for news titles
title_results = soup.find_all('div', class_='content_title')

# Search for paragraph text under news titles
p_results = soup.find_all('div', class_='article_teaser_body')

# Extract first title and paragraph, and assign to variables
news_title = title_results[0].text
news_p = p_results[0].text

print(news_title)
print(news_p)

Small Satellite Mission of the Year
The first interplanetary CubeSats were recognized by the engineering community with the 2019 Small Satellite Mission of the Year award.


### Mars Featured Imaged

In [45]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [64]:
html = browser.html
soup = bs(html, 'html.parser')

# Search for featured Mars images
results = soup.find_all('img', class_='thumb')

# Find image path to first featured image
relative_img_path = img[0]['src']

# Get full path for featured image
featured_img_url = 'https://www.jpl.nasa.gov' + relative_img_path

print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23384-640x350.jpg


### Mars Weather

In [65]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [73]:
html = browser.html
soup = bs(html, 'html.parser')

# Search for tweets
results = soup.find_all('p', class_="TweetTextSize")

# Grab text of first tweet
mars_weather = results[0].text #strip linke breaks and link

print(mars_weather)

InSight sol 250 (2019-08-10) low -100.0ºC (-148.1ºF) high -26.2ºC (-15.1ºF)
winds from the SSE at 4.4 m/s (9.8 mph) gusting to 16.2 m/s (36.2 mph)
pressure at 7.60 hPapic.twitter.com/9sZRRUi3dm


### Mars Facts